In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp v2.nixtla_client

# V2 Nixtla Client

In [ ]:
#| export
import math
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import chain
from typing import Any, List, Literal, Mapping, Optional, Union

import httpx
import numpy as np
import pandas as pd
import utilsforecast.processing as ufp
from tqdm.auto import tqdm
from utilsforecast.compat import DataFrame, pl_DataFrame, pl_Series
from utilsforecast.validation import validate_format, validate_freq

from nixtla.core.http_client import HttpClient

In [ ]:
#| exporti
_LOSS = Literal["default", "mae", "mse", "rmse", "mape", "smape"]
_MODEL = Literal["timepgt-1", "timegpt-1-long-horizon"]

In [ ]:
#| export
class NixtlaClient:

    def __init__(
        self,
        api_key: Optional[str] = None,
        base_url: Optional[str] = None,
        timeout: int = 60,
        max_retries: int = 6,
    ):
        if api_key is None:
            api_key = os.environ['NIXTLA_API_KEY']
        if base_url is None:
            base_url = os.getenv('NIXTLA_BASE_URL', 'https://dashboard.nixtla.io/api')
        self._client_kwargs = {
            'base_url': base_url,
            'headers': {'Authorization': f'Bearer {api_key}'},
            'timeout': timeout,
        }
        self.max_retries = max_retries

    def _partition_series(
        self, payload: Mapping[str, Any], n_part: int, h: int
    ) -> List[Mapping[str, Any]]:
        parts = []
        series = payload.pop('series')
        n_series = len(series['sizes'])
        n_part = min(n_part, n_series)
        series_per_part = math.ceil(n_series / n_part)
        prev_size = 0
        for i in range(0, n_series, series_per_part):
            sizes = series['sizes'][i : i + series_per_part]
            curr_size = sum(sizes)
            part_idxs = slice(prev_size, prev_size + curr_size)
            prev_size += curr_size
            part_series = {
                'y': series['y'][part_idxs],
                'sizes': sizes,
            }
            if series['X'] is None:
                part_series['X'] = None
                part_series['X_future'] = None
            else:
                part_series['X'] = [x[part_idxs] for x in series['X']]
                part_series['X_future'] = [x[i * h : (i + series_per_part) * h] for x in series['X_future']]
            parts.append({'series': part_series, **payload})
        return parts

    def forecast(
        self,
        df: DataFrame,
        h: int,
        freq: str,    
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        X_df: Optional[DataFrame] = None,
        level: Optional[List[Union[int, float]]] = None,
        quantiles: Optional[List[float]] = None,
        finetune_steps: int = 0,
        finetune_loss: _LOSS = 'default',
        clean_ex_first: bool = True,
        validate_api_key: bool = False,
        add_history: bool = False,
        date_features: Union[bool, List[str]] = False,
        model: _MODEL = 'timegpt-1',
        num_partitions: Optional[int] = None,
    ):
        """Forecast your time series using TimeGPT.

        Parameters
        ----------
        df : pandas.DataFrame
            The DataFrame on which the function will operate. Expected to contain at least the following columns:
            - id_col:
                Column name in `df` that identifies unique time series. Each unique value in this column
                corresponds to a unique time series.
            - time_col:
                Column name in `df` that contains the time indices of the time series. This is typically a datetime
                column with regular intervals, e.g., hourly, daily, monthly data points.
            - target_col:
                Column name in `df` that contains the target variable of the time series, i.e., the variable we 
                wish to predict or analyze.
        h : int
            Forecast horizon.
        freq : str
            Frequency of the data. By default, the freq will be inferred automatically.
            See [pandas' available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        X_df : pandas.DataFrame, optional (default=None)
            DataFrame with [`unique_id`, `ds`] columns and `df`'s future exogenous.
        level : List[float], optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        quantiles : List[float], optional (default=None)
            Quantiles to forecast, list between (0, 1).
            `level` and `quantiles` should not be used simultaneously.
            The output dataframe will have the quantile columns
            formatted as TimeGPT-q-(100 * q) for each q.
            100 * q represents percentiles but we choose this notation
            to avoid having dots in column names.
        finetune_steps : int (default=0)
            Number of steps used to finetune learning TimeGPT in the
            new data.
        finetune_loss : str (default='default')
            Loss function to use for finetuning. Options are: `default`, `mae`, `mse`, `rmse`, `mape`, and `smape`.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        validate_api_key : bool (default=False)
            If True, validates api_key before 
            sending requests.
        add_history : bool (default=False)
            Return fitted values of the model.
        date_features : bool or list of str or callable, optional (default=False)
            Features computed from the dates. 
            Can be pandas date attributes or functions that will take the dates as input.
            If True automatically adds most used date features for the 
            frequency of `df`.
        model : str (default='timegpt-1')
            Model to use as a string. Options are: `timegpt-1`, and `timegpt-1-long-horizon`. 
            We recommend using `timegpt-1-long-horizon` for forecasting 
            if you want to predict more than one seasonal 
            period given the frequency of your data.
        num_partitions : int (default=None)
            Number of partitions to use.
            If None, the number of partitions will be equal
            to the available parallel resources in distributed environments.
        
        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with TimeGPT forecasts for point predictions and probabilistic
            predictions (if level is not None).
        """
        if add_history:
            raise NotImplementedError('add_history=True not yet supported')
        if quantiles is not None:
            raise NotImplementedError('quantiles!=None not yet supported')
        validate_format(df=df, id_col=id_col, time_col=time_col, target_col=target_col)
        validate_freq(times=df[time_col], freq=freq)
        processed = ufp.process_df(
            df=df, id_col=id_col, time_col=time_col, target_col=target_col
        )
        if processed.data.shape[1] > 1:
            X = processed.data[:, 1:].T.tolist()
            if X_df is None:
                raise ValueError(
                    'Training exogenous features were provided but no future values.'
                )
            processed_X = ufp.process_df(
                df=X_df, id_col=id_col, time_col=time_col, target_col=None,
            )
            X_future = processed_X.data.T.tolist()
            x_cols = [c for c in df.columns if c not in (id_col, time_col, target_col)]
        else:
            X = None
            X_future = None
            x_cols = None
        payload = {
            'series': {
                'y': processed.data[:, 0].tolist(),
                'sizes': np.diff(processed.indptr).tolist(),
                'X': X,
                'X_future': X_future,
            },
            'model': model,
            'h': h,
            'freq': freq,
            'clean_ex_first': clean_ex_first,
            'level': level,
            'finetune_steps': finetune_steps,
            'finetune_loss': finetune_loss,
        }
        with httpx.Client(**self._client_kwargs) as httpx_client:
            client = HttpClient(httpx_client=httpx_client)
            if num_partitions is None:
                resp = client.request(
                    method='post',
                    url='/v2/forecast',
                    json=payload,
                    max_retries=self.max_retries,
                ).json()
            else:
                payloads = self._partition_series(payload, num_partitions, h)
                results = [None for _ in payloads]
                with ThreadPoolExecutor(min(10, num_partitions)) as executor:
                    future2pos = {
                        executor.submit(
                            client.request,
                            method='post',
                            url='/v2/forecast',
                            json=payload,
                            max_retries=self.max_retries,
                        ): i
                        for i, payload in enumerate(payloads)
                    }
                    for future in tqdm(as_completed(future2pos), total=len(future2pos)):
                        pos = future2pos[future]
                        results[pos] = future.result().json()
                resp = {'mean': list(chain.from_iterable(res['mean'] for res in results))}
                if payload['level'] is None:
                    resp['intervals'] = None
                else:
                    resp['intervals'] = {}
                    for lvl in payload['level']:
                        for side in ['lo', 'hi']:
                            resp['intervals'][f'{side}-{lvl}'] = list(
                                chain.from_iterable(
                                    res['intervals'][f'{side}-{lvl}'] for res in results
                                )
                            )
                if X is None:
                    resp['weights_x'] = None
                else:
                    resp['weights_x'] = [res['weights_x'] for res in results]
        if isinstance(df, pd.DataFrame):
            times = pd.Index(processed.times)
        elif isinstance(df, pl_DataFrame):
            times = pl_Series(processed.times)
        out = ufp.make_future_dataframe(
            uids=processed.uids,
            last_times=times,
            freq=freq,
            h=h,
            id_col=id_col,
            time_col=time_col,
        )
        out = ufp.assign_columns(out, 'TimeGPT', resp['mean'])
        if resp['intervals'] is not None:
            for lvl, vals in resp['intervals'].items():
                out = ufp.assign_columns(out, f'TimeGPT-{lvl}', vals)
        if resp['weights_x'] is not None:
            if isinstance(resp['weights_x'], list):
                self.weights_x = [
                    type(df)({
                        'features': x_cols,
                        'weights': weights_x
                    })
                    for weights_x in resp['weights_x']
                ]
            else:
                self.weights_x = type(df)({
                    'features': x_cols,
                    'weights': resp['weights_x'],
                })
        return out

In [ ]:
from utilsforecast.data import generate_series

import polars as pl

In [ ]:
series = generate_series(4, min_length=91, max_length=91)
series['unique_id'] = series['unique_id'].astype('int64')
series_pl = pl.from_pandas(series)

In [ ]:
client = NixtlaClient(max_retries=0)

In [ ]:
client.forecast(df=series, h=2, freq="D", level=[80])

,unique_id,ds,TimeGPT,TimeGPT-lo-80,TimeGPT-hi-80
0,0,2000-04-01,1.245500,0.149068,2.341933
1,0,2000-04-02,1.378139,1.033565,1.722714
2,1,2000-04-01,1.224127,0.013970,2.434283
3,1,2000-04-02,1.407930,1.109384,1.706476
4,2,2000-04-01,1.379222,0.126348,2.632095
5,2,2000-04-02,1.532854,1.188261,1.877446
6,3,2000-04-01,1.225660,0.100382,2.350937
7,3,2000-04-02,1.418851,1.160756,1.676946


In [ ]:
client.forecast(df=series, h=2, freq="D", level=[80], num_partitions=2)

100%|█████████████████████████████████████████████| 2/2 [00:06<00:00,  3.16s/it]


,unique_id,ds,TimeGPT,TimeGPT-lo-80,TimeGPT-hi-80
0,0,2000-04-01,1.245500,0.057620,2.433381
1,0,2000-04-02,1.378139,1.033564,1.722714
2,1,2000-04-01,1.224127,0.020008,2.428245
3,1,2000-04-02,1.407930,1.142118,1.673742
4,2,2000-04-01,1.379222,0.150685,2.607758
5,2,2000-04-02,1.532854,1.223727,1.841980
6,3,2000-04-01,1.225660,0.035012,2.416308
7,3,2000-04-02,1.418851,1.130188,1.707515


In [ ]:
client.forecast(df=series_pl, h=2, freq="1d", level=[80])

unique_id,ds,TimeGPT,TimeGPT-lo-80,TimeGPT-hi-80
i64,datetime[ns],f64,f64,f64
0,2000-04-01 00:00:00,1.2455,0.149068,2.341933
0,2000-04-02 00:00:00,1.378139,1.033565,1.722714
1,2000-04-01 00:00:00,1.224127,0.01397,2.434283
1,2000-04-02 00:00:00,1.40793,1.109384,1.706476
2,2000-04-01 00:00:00,1.379222,0.126348,2.632095
2,2000-04-02 00:00:00,1.532854,1.188261,1.877446
3,2000-04-01 00:00:00,1.22566,0.100382,2.350937
3,2000-04-02 00:00:00,1.418851,1.160756,1.676946


In [ ]:
from nixtla import NixtlaClient as NixtlaV1

In [ ]:
v1_client = NixtlaV1()

In [ ]:
v1_client.forecast(df=series, h=2, freq="D", level=[80])

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,unique_id,ds,TimeGPT,TimeGPT-lo-80,TimeGPT-hi-80
0,0,2000-04-01,1.245500,0.149068,2.341933
1,0,2000-04-02,1.378139,1.033565,1.722714
2,1,2000-04-01,1.224127,0.013970,2.434283
3,1,2000-04-02,1.407930,1.109384,1.706476
4,2,2000-04-01,1.379222,0.126348,2.632095
5,2,2000-04-02,1.532854,1.188261,1.877446
6,3,2000-04-01,1.225660,0.100382,2.350937
7,3,2000-04-02,1.418851,1.160756,1.676946


In [ ]:
v1_client.forecast(df=series, h=2, freq="D", level=[80], num_partitions=2)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,unique_id,ds,TimeGPT,TimeGPT-lo-80,TimeGPT-hi-80
0,0,2000-04-01,1.245500,0.057620,2.433381
1,0,2000-04-02,1.378139,1.033564,1.722714
2,1,2000-04-01,1.224127,0.020008,2.428245
3,1,2000-04-02,1.407930,1.142118,1.673742
4,2,2000-04-01,1.379222,0.150685,2.607758
5,2,2000-04-02,1.532854,1.223727,1.841980
6,3,2000-04-01,1.225660,0.035012,2.416308
7,3,2000-04-02,1.418851,1.130188,1.707515
